# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
#import gmaps.datasets
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
#read csv for dataframe
weather_csv = "../output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,rikitea,14,PF,1666724630,75,-23.1203,-134.9692,72.81,19.30
1,1,albany,98,US,1666724597,90,42.6001,-73.9662,71.26,3.00
2,2,lompoc,28,US,1666724455,60,34.6391,-120.4579,77.85,4.00
3,3,grand gaube,100,US,1666724631,35,38.8755,-109.5012,47.95,1.70
4,4,busselton,19,AU,1666724631,86,-33.6500,115.3333,51.76,1.81


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
gmaps.configure(api_key=g_key)

#drop null values
weather_df = weather_df.dropna()

#Locations and Humidity
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)

#make figure and heatmap
fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                              max_intensity=max(weather_df["Humidity"]))
fig.add_layer(heatmap)

fig

plt.savefig("../Images/heatmap.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
#new dataframe for filtered data
filtered_weather_df = weather_df

#filtering the data
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Humidity"] > 60]

#drop null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
76,76,puerto ayora,0,ES,1666724663,82,36.5939,-6.2330,73.26,4.61
227,227,surt,0,LY,1666724729,68,31.2089,16.5887,74.57,7.45
254,254,veraval,0,IN,1666724738,68,20.9000,70.3667,78.33,8.03
362,362,mahajanga,0,MG,1666724798,78,-15.7167,46.3167,78.58,9.71
369,369,puerto leguizamo,0,ES,1666724744,82,36.5939,-6.2330,73.26,4.61
423,423,marsa matruh,0,MT,1666724825,88,35.8792,14.4953,73.27,3.44
475,475,mareeba,0,AU,1666724852,91,-17.0000,145.4333,74.44,2.71
483,483,puerto suarez,0,ES,1666724663,82,36.5939,-6.2330,73.26,4.61
517,517,puerto narino,0,ES,1666724872,82,36.5939,-6.2330,73.26,4.61
527,527,salalah,0,OM,1666724875,61,17.0151,54.0924,77.09,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = filtered_weather_df

# params dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # change location while leaving params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make api request
    response = requests.get(url, params=params).json()

    results = response['results']
    
    try:
        print(f"For Index {index}: {row['City']}, the closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing")

For Index 76: puerto ayora, the closest hotel is Hotel Campomar Playa.
For Index 227: surt, the closest hotel is فندق المدينة.
For Index 254: veraval, the closest hotel is Lords Inn Somnath.
For Index 362: mahajanga, the closest hotel is Hôtel Restaurant Gourmand Coco Lodge Majunga.
For Index 369: puerto leguizamo, the closest hotel is Hotel Campomar Playa.
For Index 423: marsa matruh, the closest hotel is Grand Hotel Excelsior Malta.
For Index 475: mareeba, the closest hotel is Trinity Plains Tourist Park.
For Index 483: puerto suarez, the closest hotel is Hotel Campomar Playa.
For Index 517: puerto narino, the closest hotel is Hotel Campomar Playa.
For Index 527: salalah, the closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.


In [31]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
76,76,puerto ayora,0,ES,1666724663,82,36.5939,-6.2330,73.26,4.61,Hotel Campomar Playa
227,227,surt,0,LY,1666724729,68,31.2089,16.5887,74.57,7.45,فندق المدينة
254,254,veraval,0,IN,1666724738,68,20.9000,70.3667,78.33,8.03,Lords Inn Somnath
362,362,mahajanga,0,MG,1666724798,78,-15.7167,46.3167,78.58,9.71,Hôtel Restaurant Gourmand Coco Lodge Majunga
369,369,puerto leguizamo,0,ES,1666724744,82,36.5939,-6.2330,73.26,4.61,Hotel Campomar Playa
423,423,marsa matruh,0,MT,1666724825,88,35.8792,14.4953,73.27,3.44,Grand Hotel Excelsior Malta
475,475,mareeba,0,AU,1666724852,91,-17.0000,145.4333,74.44,2.71,Trinity Plains Tourist Park
483,483,puerto suarez,0,ES,1666724663,82,36.5939,-6.2330,73.26,4.61,Hotel Campomar Playa
517,517,puerto narino,0,ES,1666724872,82,36.5939,-6.2330,73.26,4.61,Hotel Campomar Playa
527,527,salalah,0,OM,1666724875,61,17.0151,54.0924,77.09,3.44,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
plt.savefig("../Images/hotel_map2.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>